!rm -rf SSD-keras# Untitled Notebook

This is an initial placeholder notebook. Feel free to edit and rename as well as create your own notebooks, to use Google Cloud Datalab.

In [2]:
import os

os.chdir('/content')

if not os.path.exists("SSD-keras"):
    !git clone https://github.com/faithnh/SSD-keras.git -b run-tensorflow
else:
    !git pull origin run-tensorflow

os.chdir('/content/SSD-keras')

import sys
sys.path.append('src')

!ls

!pwd

Cloning into 'SSD-keras'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 3806 (delta 2), reused 0 (delta 0), pack-reused 3796
Receiving objects: 100% (3806/3806), 167.29 MiB | 31.72 MiB/s, done.
Resolving deltas: 100% (2517/2517), done.
datasets  images  LICENSE  src	test.ipynb  trained_models
/content/SSD-keras


In [0]:
!chmod +x ./datasets/VOC0712_downloader.sh

!./datasets/VOC0712_downloader.sh

!mv VOCdevkit ./datasets/

datasets  images  LICENSE  src	test.ipynb  trained_models


In [25]:
import os

from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import SGD

from models import SSD300
from utils.boxes import create_prior_boxes, to_point_form
from utils.data_management import DataManager, get_class_names
from utils.data_generator import DataGenerator
from utils.training import MultiboxLoss, scheduler


model_name = 'SSD300_VOC2007'
# hyper-parameters
batch_size = 2
num_epochs = 250
alpha_loss = 1.0
learning_rate = 1e-3
momentum = .9
weight_decay = 5e-4
gamma_decay = 0.1
# scheduled_epochs = [155, 195, 235]
negative_positive_ratio = 3
base_weights_path = './trained_models/VGG16_weights.hdf5'

# data
class_names = get_class_names('VOC2007')
val_dataset, val_split = 'VOC2007', 'test'
train_datasets, train_splits = ['VOC2007', 'VOC2012'], ['trainval', 'trainval']
train_data_manager = DataManager(
    dataset_name=train_datasets,
    split=train_splits,
    class_names=class_names,
    dataset_path="./datasets/VOCdevkit/"
)
train_data = train_data_manager.load_data()
num_classes = len(class_names)
val_data_manager = DataManager(
    dataset_name=val_dataset,
    split=val_split,
    class_names=class_names,
    dataset_path="./datasets/VOCdevkit/"
)
val_data = val_data_manager.load_data()

# model
model = SSD300(num_classes=num_classes)
if os.path.exists(base_weights_path):
  model.load_weights(base_weights_path, by_name=True)

prior_boxes = to_point_form(create_prior_boxes())
multibox_loss = MultiboxLoss(num_classes, negative_positive_ratio, alpha_loss)
optimizer = SGD(learning_rate, momentum, weight_decay)
model.compile(optimizer, loss=multibox_loss.compute_loss)
data_generator = DataGenerator(
        train_data, prior_boxes, batch_size, num_classes, val_data)

print(data_generator.flow(mode='train').__next__())

# # callbacks
# model_path = '../trained_models/' + model_name + '/'
# save_path = model_path + 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
# if not os.path.exists(model_path):
#     os.makedirs(model_path)
# log = CSVLogger(model_path + model_name + '.log')
# checkpoint = ModelCheckpoint(save_path, verbose=1, save_weights_only=True)
# # reduce_on_plateau = ReduceLROnPlateau(factor=gamma_decay, verbose=1)
# # scheduler = LearningRateManager(learning_rate, gamma_decay, scheduled_epochs)
# learning_rate_schedule = LearningRateScheduler(scheduler, verbose=1)
# callbacks = [checkpoint, log, learning_rate_schedule]
# # callbacks = [checkpoint, log, reduce_on_plateau]

# # training
# model.summary()
# model.fit_generator(data_generator.flow(mode='train'),
#                     steps_per_epoch=int(len(train_data) / batch_size),
#                     epochs=num_epochs,
#                     verbose=1,
#                     callbacks=callbacks,
#                     validation_data=data_generator.flow(mode='val'),
#                     validation_steps=int(len(val_data) / batch_size),
#                     use_multiprocessing=False,
#                     workers=1)

[{'input_1': array([[[[ 6.58148956e+01,  1.01960373e+01,  2.92087402e+01],
         [ 6.20205841e+01,  7.33277130e+00,  2.63454590e+01],
         [ 5.83045807e+01,  5.00635529e+00,  2.40190430e+01],
         ...,
         [-2.20594482e+01, -2.92578812e+01, -3.34166794e+01],
         [-1.70335693e+01, -2.30380859e+01, -3.12035828e+01],
         [-1.02185059e+01, -1.25142365e+01, -2.80918198e+01]],

        [[ 6.60181885e+01,  1.03993378e+01,  2.94120331e+01],
         [ 6.26366272e+01,  7.94882202e+00,  2.69615173e+01],
         [ 5.83045807e+01,  5.00634766e+00,  2.40190430e+01],
         ...,
         [-1.90031662e+01, -2.60244293e+01, -3.03249664e+01],
         [-1.15944519e+01, -1.67478714e+01, -2.65004044e+01],
         [-2.04720383e+01, -2.10102234e+01, -3.91775513e+01]],

        [[ 6.57424469e+01,  1.01235886e+01,  2.91362915e+01],
         [ 6.19034576e+01,  7.21563721e+00,  2.62283325e+01],
         [ 5.82061462e+01,  4.90792084e+00,  2.39206085e+01],
         ...,
         [-

src/utils/boxes.py:65: RuntimeWarning: divide by zero encountered in divide
  encoded_center_x = encoded_center_x / (x_min_scale * w_prior)
src/utils/boxes.py:67: RuntimeWarning: divide by zero encountered in divide
  encoded_center_y = encoded_center_y / (y_min_scale * h_prior)
src/utils/boxes.py:69: RuntimeWarning: divide by zero encountered in divide
  encoded_w = (x_max - x_min) / w_prior
src/utils/boxes.py:71: RuntimeWarning: divide by zero encountered in divide
  encoded_h = (y_max - y_min) / h_prior
